In [239]:
# This notebook will be mainly used for the capstone project

import pandas as pd
import numpy as np

print('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [240]:
# Reading table from the webpage
toron2 = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
toron = toron2[0]
toron.head(5)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [241]:
# Get names of indexes for which borough is not been assigned
indexNames = toron[ toron['Borough'] == 'Not assigned' ].index
 
# Delete these row indexes from dataFrame
toron.drop(indexNames , inplace=True)

# and reset index
toron = toron.reset_index(drop=True)
toron.head(15)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [242]:
# this will replace the neighbourhood as borough (when neighbourhood is not been assigned)
for ind in toron.index: 
    if toron['Neighbourhood'][ind] == 'Not assigned':
        toron['Neighbourhood'][ind] = toron['Borough'][ind]
    
toron.head(15)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [243]:
# combining rows so that neighborhoods with the same postalcode appear together separated by a comma
toron = toron.groupby(['Postcode','Borough'])['Neighbourhood'].apply(list)
toron = toron.sample(frac=1).reset_index()
toron['Neighbourhood']= toron['Neighbourhood'].str.join(', ') 
toron.head(15)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4G,East York,Leaside
2,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi..."
3,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
4,M5M,North York,"Bedford Park, Lawrence Manor East"
5,M1S,Scarborough,Agincourt
6,M5C,Downtown Toronto,St. James Town
7,M5S,Downtown Toronto,"Harbord, University of Toronto"
8,M9P,Etobicoke,Westmount
9,M3M,North York,Downsview Central


In [244]:
toron.shape

(103, 3)